# Chương trình tính dự báo rating dữ liệu nạp trực tiếp từ file .csv

# Sử dụng 2 độ đo tương tự Cosin và Pearson

In [2]:
import math
import csv
import pandas as pd


## Nạp dữ liệu và tính toán ban đầu

In [ ]:
Ratings = pd.read_csv("u1.base", encoding="utf-8", sep='\t', header=None, names=['userId','movieId','rating','tmp'], engine='python')
del Ratings['tmp']

#Tinh trung bình rating của các user

r_avg = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
dmean = {}
for i in range(len(r_avg)):
    uid = r_avg.iloc[i].userId
    rating = r_avg.iloc[i].rating
    dmean[uid] = rating

# Tinh d1 là từ điển chứa các rating của từng user: {1: {1: 5, 2: 3},...} la user 1 rating movie 1 mức 5,2 mức 3,...
d1 = {} 
for i in range(len(Ratings)):
    uid = Ratings.iloc[i].userId
    mid = Ratings.iloc[i].movieId
    rating = Ratings.iloc[i].rating
    if uid in d1:
        p = d1.get(uid)
        p[mid] = rating 
    else:
        p={mid:rating}
        d1[uid] = p

# # Dự đoán rating bằng độ đo Cosin

In [5]:
def sim_cosin(r1, r2):
    t = 0
    m1 = 0
    m2 = 0
    for i in range(len(r1)):
        t = t + r1[i]*r2[i]
        m1 = m1 + r1[i]*r1[i]
        m2= m2+ r2[i]*r2[i]
    return t/math.sqrt(m1*m2)

def predictRatingCosin(user, movie):
    l1 = d1.get(user).keys() #nhung movie user rated
    lu = [] # nhung user co chung rating it nhat 1 movie 
    for i in d1:
        li = d1.get(i).keys() #Nhung movie ma user i rated
        if i!=user and len(l1&li) > 5:
            lu.append(i) #them user i vao ds chung
    
    sim = [] #tinh do tuong tu giua user voi cac user có chung rated
    for i in lu:
        r1=[]
        r2=[]
        l2 = d1.get(i).keys()
        l = l1&l2
        for j in l:
            r1.append(d1[user][j])
            r2.append(d1[i][j])
        sim.append(sim_cosin(r1,r2))
    t = 0
    m = 0
    for i in range(len(lu)):
        u2 = lu[i]
        if movie in d1[u2] and sim[i] > 0.5:
            t = t + (d1[u2][movie])*sim[i]
            m = m + sim[i]
    return t/m        

3.6957641951374014


## Dự đoán rating bằng độ đo Pearson

In [4]:
def sim_pearson(r1, r2, av1, av2):
    t = 0
    m1 = 0
    m2 = 0
    for i in range(len(r1)):
        t = t + (r1[i]-av1)*(r2[i]-av2)
        m1 = m1 + (r1[i]-av1)*(r1[i]-av1)
        m2= m2+ (r2[i]-av2)*(r2[i]-av2)
    return t/(math.sqrt(m1)*math.sqrt(m2))

def predictRatingPearsonx(user, movie):
    l1 = d1.get(user).keys() #nhung movie user rated
    lu = [] # nhung user co chung rating it nhat 1 movie 
    for i in d1:
        li = d1.get(i).keys() #Nhung movie ma user i rated
        if i!=user and len(l1&li) > 5:
            lu.append(i) #them user i vao ds chung
    
    sim = [] #tinh do tuong tu giua user voi cac user có chung rated
    for i in lu:
        r1=[]
        r2=[]
        l2 = d1.get(i).keys()
        l = l1&l2
        for j in l:
            r1.append(d1[user][j])
            r2.append(d1[i][j])
        sim.append(sim_pearson(r1,r2, dmean[user],dmean[i]))
    t = 0
    m = 0
    for i in range(len(lu)):
        u2 = lu[i]
        if movie in d1[u2] and sim[i] > 0.1:
            t = t + (d1[u2][movie]-dmean[u2])*sim[i]
            m = m + sim[i]
    return dmean[user]+t/m        